In [1]:
import pytz
import datetime
import pandas as pd
import glob
import os
import numpy as np
import gc

In [ ]:
# Initiating

df_total_mapping=pd.DataFrame(columns=['Customer_Link','customer_id_hashed','date_up_to'])
print("Start",datetime.datetime.now())

for ind, row in df_all_mapping_file.iterrows():
    file_path=row['file_path']
    date_up_to=row['mapping_up_to_date']
    df=pd.read_table(file_path,dtype=str,sep="|",compression="gzip",usecols=['Customer_Link','customer_id_hashed'])
    df=df[df['Customer_Link']!="UNMATCHED"].drop_duplicates("Customer_Link")
    df['date_up_to']=date_up_to
    df_pre=df_total_mapping.drop_duplicates("Customer_Link") # as the high date in the cum mapping
    df_pre['latest']=1 # label as the latest avaibale
    df_total_mapping=pd.merge(df_total_mapping,df_pre,on=["Customer_Link","customer_id_hashed","date_up_to"],how="left")
    df_total_mapping=df_total_mapping[pd.isnull(df_total_mapping['latest'])] # remove the latest and kept previous only if different
    del df_pre['latest']
    del df_total_mapping['latest']
    df_2_consecutive=df.append(df_pre) # new on the top, old below
    
    df_ids_count_by_idl=df_2_consecutive.groupby("Customer_Link")["customer_id_hashed"].nunique().to_frame().reset_index()
    df_ids_count_by_idl=df_ids_count_by_idl.rename(columns={"customer_id_hashed":"BL_id_count"})
    df_ids_count_by_idl_2=df_ids_count_by_idl[df_ids_count_by_idl['BL_id_count']>1] # the new diffrent from the old high date
    df_ids_count_by_idl_1=df_ids_count_by_idl[df_ids_count_by_idl['BL_id_count']==1]# the new same as the old high date
    # 1 and 2 should be mutually exclusive
    df_1=pd.merge(df_2_consecutive,df_ids_count_by_idl_1,on="Customer_Link",how="inner").drop_duplicates("Customer_Link") # keep the new only
    df_2=pd.merge(df_2_consecutive,df_ids_count_by_idl_2,on="Customer_Link",how="inner") # keep both
    
    del df_1['BL_id_count']
    del df_2['BL_id_count']
    
    print(df_1.columns.tolist())
    print(df_2.columns.tolist())
    
    df_total_mapping=pd.concat([df_1,df_2,df_total_mapping])
    
    print(ind,datetime.datetime.now())
print("Done",datetime.datetime.now())
del df_1
del df_2
del df_2_consecutive
del df_ids_count_by_idl
del df_ids_count_by_idl_1
del df_ids_count_by_idl_2
del df
gc.collect()
df_total_mapping=df_total_mapping.sort_values(['Customer_Link','date_up_to'],ascending=[True,False])
gc.collect()
df_total_mapping.shape

In [ ]:
print(df_total_mapping.shape)
print(df_total_mapping['Customer_Link'].nunique())
print(df_total_mapping['customer_id_hashed'].nunique())
df_total_mapping.head(10)

'''
(38226704, 3)
36944095
32575028
'''

'''
Customer_Link	customer_id_hashed	date_up_to
24652785	XY1468---077tVclzugrAPADjYKKd1D3ZebCh26abPI1mraTs	c038e1b89f0bf1902d4c365d001b9a19b3a739358d1ee7...	20200215
35417849	XY1468---0pTSu_zfDGheJ-qYQPfvwhCI1e7f_joTNdovjomE	1b5cda9e5ca0af148177d6a317905639cb35d744e0ff2d...	20200215
2516469	XY1468---1i3EbLRBAPuKc2mSWy4bIs6QvBbFwWavuATdTTQI	0d95ec6248ba4fb596378678e36c1a0453b8ff84862b76...	20200215
17982002	XY1468---1kVQT3iPqthySc6GJ53sp2-h3GwT4_4bYAJk__eI	d449269735be97a927b7236184082861ecd4a74810e181...	20200215
36051352	XY1468---2HCWhlYC73t__zvlTPujqfK2YR9dz6MAg64po5hM	96457a5c6aeb15284a930557afdf3032933a1510257b62...	20200215
57126037	XY1468---2Lpewd2wQcUpWwH177TLrtS7CY2j703yOv6MJ6Lk	0fe3858f393b63931a48cb0a11665cc15e78869df39db7...	20190207
47935399	XY1468---2X7UQqC1vASJx-IeoLZoXp3vxApZpE6um4fesQsU	5c701b9026542480dc97b4535bf751b665106396f9a6b3...	20191125
15797510	XY1468---2eCOjc0YWoD7YeA3caYqs9co13bn5GbXVVsLNKhs	e7599bf13053e3a3b444da6330ff1cff49579c9ea53c73...	20200215
6716466	XY1468---2yhYvOTg1BNenE00-7L1WPENGOjZhJ5YVd8bKzTE	21ad9af4897aece7f81910a8e5bf7d4f92409f31c734e0...	20200215
13451377	XY1468---4MVMx_4NhiHNoYG0Ee9uHc0JtD9JochFalgSMIbw	d3f7f025574ae8fd84aa451c6a81d4c93da5c835540b17...	20200215
'''

In [ ]:
2020-03-02

df_total_mapping.to_csv("./df_total_mapping_IDL_BLid_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
print(datetime.datetime.now())

